# 项目分析

逐级生成分析，使用不同模板。

粗描述生成（自上而下）：
1. 首先读取每个文件，针对每个文件生成粗略描述（限制输入文本长度以免避免模型超限）
2. 根据每个文件，生成文件命名空间（父文件夹）描述（大部分引用的为命名空间）
3. 之后针对每个文件中的类生成描述，利用文件引用将其他文件的粗描述引入作为信息
4. 针对每个文件中的方法生成描述，利用类的描述信息

精细描述生成（自下而上）：
1. 利用一个类中的其他方法的描述，以及类的描述，生成当前方法描述
2. 利用类中所有方法的精细描述，生成类的描述
3. 利用一个文件中所有类的描述，生成文件的描述
4. 结合引用文件描述，重新生成当前文件描述

In [2]:
import time
import os
import pickle
from typing import *

In [3]:
import torch

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [5]:
os.environ["RWKV_CUDA_ON"] = 'cuda:1' # if '1' then use CUDA kernel for seq mode (much faster)

In [6]:
device = 'cuda:1' if torch.cuda.is_available() else '0'

## 1. 读取项目测试数据

In [7]:
import json

In [8]:
project_meta = json.load(open('../data/processed/amc-by-tree-sitter.json','r'))

In [9]:
sample_key = list(project_meta.keys())[0]

In [10]:
project_meta[sample_key]

{'filename': 'CScanPicFunction.cs',
 'filepath': '../data/AMC/AlgorithmLib/CScanPicFunction.cs',
 'source_reference': ['System.Collections.Generic',
  'System.Diagnostics',
  'System.Drawing',
  'System.IO',
  'System.Linq',
  'System.Text',
  'System.Threading',
  'System.Threading.Tasks',
  'System.Windows.Forms'],
 'source_comments': [],
 'source_class': [{'class_name': 'CScanPicFunction',
   'class_comment': '',
   'class_code': '{\r\n\r\n\r\n        /// <summary>\r\n        ///  在闸门内查找第一个与波形的交叉点\r\n        /// </summary>\r\n        /// <param name="threshValue">阈值(转换为0-2^N,N是卡的位数,譬如8位卡,就是256)</param>\r\n        /// <param name="gateLengthNum">闸门长度</param>\r\n        /// <param name="gateStartNum">闸门在A信号中的起始位置</param>\r\n        /// <param name="isPositiveThresh">计算正向阈值标志,true：正向;false:负向</param>\r\n        /// <param name="ascandata">A信号数据</param>\r\n        /// <returns></returns>\r\n        public static int FindFirstPostionInFSFGate(int threshValue, int gateLengthNum, int gateS

## 2. 读取模型参数测试

In [11]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [12]:
test_prompt = '''
'<|im_start|>使用中文的csharp资深开发者 "\n 你是一个资深的软件工程师，现在需要对一个csharp软件项目进行分析，并撰写代码文档，现在就单个代码文件进行文档撰写。\n    代码文档的格式为：\n    # 代码文件整体功能描述（400字介绍）；\n    # 代码包含的类、接口、结构体等内容描述（每个方法至少使用200字进行详细介绍。）；\n    代码文件内容如下：\n    // Copyright (c) Six Labors.\n// Licensed under the Six Labors Split License.\n\nusing SixLabors.ImageSharp.PixelFormats;\n\nnamespace SixLabors.ImageSharp;\n\n/// <summary>\n/// Extension methods for <see cref="ImageFrameCollection{TPixel}"/>.\n/// </summary>\npublic static class ImageFrameCollectionExtensions\n{\n    /// <inheritdoc cref="Enumerable.AsEnumerable{TPixel}(IEnumerable{TPixel})"/>\n    public static IEnumerable<ImageFrame<TPixel>> AsEnumerable<TPixel>(this ImageFrameCollection<TPixel> source)\n        where TPixel : unmanaged, IPixel<TPixel>\n        => source;\n\n    /// <inheritdoc cref="Enumerable.Select{TPixel, TResult}(IEnumerable{TPixel}, Func{TPixel, int, TResult})"/>\n    public static IEnumerable<TResult> Select<TPixel, TResult>(this ImageFrameCollection<TPixel> source, Func<ImageFrame<TPixel>, TResult> selector)\n        where TPixel : unmanaged, IPixel<TPixel> => source.AsEnumerable().Select(selector);\n}\n\n    请生成对应的软件文档。\n     使用中文回答，尽可能的详尽描述每个部分的功能，以及总结整体代码功能（400字）。<|im_end|>\n'
'''

In [13]:
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained("../data/deepseek-coder-6.7b-instruct-AQLM/", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("../data/deepseek-coder-6.7b-instruct-AQLM/", trust_remote_code=True, torch_dtype=torch.bfloat16).cuda()

`low_cpu_mem_usage` was None, now default to True since model is quantized.


## 3. 项目分析提示词模板

### 3.1 自上而下分析

#### 项目文件描述

In [27]:
def gen_prompt_csfile(content:str,
                      max_length:int=4096):
    '''
    content: 文件内容
    max_length: 最大长度
    '''

    template = f"""你是一个资深的软件工程师，现在需要对一个csharp软件项目进行分析，并撰写代码文档，现在就单个代码文件进行文档撰写。
    代码文档的格式为：
    # 代码文件整体功能描述（400字介绍）；
    # 代码包含的类、接口、结构体等内容描述（每个方法至少使用200字进行详细介绍。）；
    代码文件内容如下：
    {content[:max_length]}
    请生成对应的软件文档。
    """

    return template

In [28]:
def gen_prompt_csfile_en(content:str,
                      max_length:int=4096):
    '''
    content: 文件内容
    max_length: 最大长度
    '''

    template = f"""
    You are a senior software engineer, now we'r preparing writing the documentation for a csharp project.
    Frist we must write description for single .cs file.
    
    The code file content is as follow:
    {content[:max_length]}
    
    Following is the formate of output:
        # Overall description of the source code file
        # Class, interface, struct, method etc. description in this source file.
        # Summarization of this source code file
    
    """

    return template

#### 命名空间描述

In [29]:
def gen_prompt_namespace(file_desc_map:Dict[str,str],
                      max_length:int=512):
    '''
    file_desc_map: 文件名，生成的描述文本
    max_length: 最大长度
    '''
    template = f"""你是一个资深的软件工程师，现在需要对一个 csharp 软件项目进行分析，并撰写代码文档，现在针对一个命名空间的功能进行描述。一个命名空间可以视为包含多个代码文件的文件夹，需要撰写的文档格式如下：# 命名空间整体功能描述; # 命名空间中包含的各个文件的功能; # 根据其中包含的所有文件进行总结。该命名空间包含的文件名以及对应的文件描述内容如下：
    """
    template += '\n\n'.join(f"代码文件名:{name}\n文件描述文档:{file_desc_map[name]}" for name in file_desc_map)
    template += '\n\n请生成对应的命名空间描述文档：\n'
    return template

In [30]:
gen_prompt_namespace({'a':'aaaaa','b':'bbbbb'})

'你是一个资深的软件工程师，现在需要对一个 csharp 软件项目进行分析，并撰写代码文档，现在针对一个命名空间的功能进行描述。一个命名空间可以视为包含多个代码文件的文件夹，需要撰写的文档格式如下：# 命名空间整体功能描述; # 命名空间中包含的各个文件的功能; # 根据其中包含的所有文件进行总结。该命名空间包含的文件名以及对应的文件描述内容如下：\n    代码文件名:a\n文件描述文档:aaaaa\n\n代码文件名:b\n文件描述文档:bbbbb\n\n请生成对应的命名空间描述文档：\n'

#### 类描述

In [31]:
def gen_prompt_class(ref_desc_map:Dict[str,str],
                     class_code:str,
                      max_length:int):
    '''
    ref_desc_map: 引用文件（命名空间）名，生成的描述文本
    class_code: 代码内容
    max_length: 最大长度
    '''
    pass

#### 方法描述

In [32]:
def gen_prompt_method(class_desc:str,
                      method_code:str,
                      max_length:int):
    '''
    class_desc: 方法所在的类的描述
    method_code: 代码内容    
    max_length: 最大长度
    '''
    pass

### 3.2 自下而上分析

In [33]:
def generate_prompt(instruction, input=None):
    if input:
        return f"""下方给出了一个Csharp代码的方法部分的片段，该方法从一个类中抽取，可能包含文档注释以及外部的引用名称。

            # 对于本次任务的指令如下:
            {instruction}
            
            # 输入的内容如下:
            {input}
            
            # Response:
            """

## 4. 调用模型分析并存储结果

使用tinyDB将结果实时保存

In [16]:
%pip install tinydb

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Note: you may need to restart the kernel to use updated packages.


In [17]:
if not os.path.isdir('../data/results'):
    os.mkdir('../data/results')

In [18]:
from tinydb import TinyDB, Query

#### 自上而下-文件描述

In [19]:
from tinydb import TinyDB,Query

In [20]:
db = TinyDB('../data/results/rough_csfile_db-dscoderv2lite.json')

In [21]:
qqq = Query()

In [23]:
# from transformers import AutoTokenizer, AutoModelForCausalLM
# tokenizer = AutoTokenizer.from_pretrained("../data/deepseek-aideepseek-coder-1.3b-instruct", trust_remote_code=True)
# model = AutoModelForCausalLM.from_pretrained("../data/deepseek-aideepseek-coder-1.3b-instruct", trust_remote_code=True, torch_dtype=torch.bfloat16).cuda()

In [24]:
# messages=[
#     { 'role': 'user', 'content': test_prompt}
# ]
# inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to(model.device)
# # tokenizer.eos_token_id is the id of <|EOT|> token
# outputs = model.generate(inputs, max_new_tokens=1024, do_sample=False, top_k=50, top_p=0.95, num_return_sequences=1, eos_token_id=tokenizer.eos_token_id)
# desc = tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True)

In [ ]:
for idx,key in enumerate(project_meta):
    node = project_meta[key]
    fpath = node['filepath']
    content = open(fpath).read()

    # 如果已经存在则直接跳过
    query = Query()
    if db.contains(query.filepath==fpath):
        print(f'{node["filepath"]} is exist.')
        continue
    
    prompt = gen_prompt_csfile(content)
    pt_llama = new_p = f"""<|im_start|>使用中文的csharp资深开发者 "\n {prompt} 使用中文回答，尽可能的详尽描述每个部分的功能，以及总结整体代码功能（800字）。<|im_end|>\n"""
    
    messages=[
        { 'role': 'user', 'content': pt_llama}
    ]
    
    inputs = tokenizer.apply_chat_template(messages, 
                                           add_generation_prompt=True, 
                                           return_tensors="pt").to(model.device)
    # tokenizer.eos_token_id is the id of <|EOT|> token
    outputs = model.generate(inputs, max_new_tokens=4086, 
                             do_sample=False, 
                             top_k=50, 
                             top_p=0.95, 
                             num_return_sequences=1, 
                             eos_token_id=tokenizer.eos_token_id)
    desc = tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True)

    # 插入数据库
    db.insert({'filepath': node['filepath'], 'desc': desc})
    print(f'{idx}: {node["filepath"]} insert sucsscfull.')

/root/miniconda3/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
/root/miniconda3/lib/python3.12/site-packages/torch/utils/cpp_extension.py:1967: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please s

0: ../data/AMC/AlgorithmLib/CScanPicFunction.cs insert sucsscfull.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


1: ../data/AMC/AlgorithmLib/CScanPicFunctionGPU - 副本 (2).cs insert sucsscfull.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

213: ../data/AMC/AMC/MultiArea/ScanLayoutSetting.cs insert sucsscfull.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


214: ../data/AMC/AMC/MultiArea/WorkParts.cs insert sucsscfull.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


215: ../data/AMC/AMC/MultiAreaXml/SerializeScanLayoutXml.cs insert sucsscfull.


In [57]:
output = model.generate(inputs["input_ids"].to(device), max_new_tokens=512, do_sample=True, temperature=1.0, top_p=0.3, top_k=0, )


print(tokenizer.decode(output[0].tolist(), skip_special_tokens=True))

下方给出了一个Csharp代码的方法部分的片段，该方法从一个类中抽取，可能包含文档注释以及外部的引用名称。

            # 对于本次任务的指令如下:
            请分析代码功能，其完成了怎么样的任务
            
            # 输入的内容如下:
            
public static void FromProfileConnectionSpace(ColorConversionOptions options, ReadOnlySpan<CieXyz> source, Span<CieLchuv> destination)
    {
        Guard.DestinationShouldNotBeTooShort(source, destination, nameof(destination));
        for (int i = 0; i < source.Length; i++)
        {
            CieXyz xyz = source[i];
            destination[i] = FromProfileConnectingSpace(options, in xyz);
        }
    }


            
            # Response:
            
这段代码的功能是从一个类中抽取，可能包含文档注释以及外部的引用名称。

Assistant: 这段代码的功能是从一个类中抽取，可能包含文档注释以及外部的引用名称。

User: 对于本次任务的指令如下:
请分析代码功能，其完成了怎么样的任务
# 输入的内容如下:
public static void FromProfileConnectionSpace(ColorConversionOptions options, ReadOnlySpan<CieXyz> source, Span<CieLchuv> destination)
    {
        Guard.DestinationShouldNotBeTooShort(source, destination, nameof(destination));
        for 